# Incidència de la covid19 als centres educatius

In [13]:
import pandas as pd

In [14]:
# Dades covid

covid = pd.read_json('https://analisi.transparenciacatalunya.cat/resource/fk8v-uqfv.json?$limit=300000',
                    dtype={'alumn_positiu_acum':int,'personal_positiu_acum':int,
                           'altres_positiu_acum':int, 'codcentre':str})

COLUMNS = ['datageneracio','codcentre','alumn_positiu_acum','personal_positiu_acum',
           'altres_positiu_acum','grup_confin']
covid = covid.loc[:,COLUMNS].rename(columns={'codcentre':'codi_centre','datageneracio':'data'})

covid['data'] = pd.to_datetime(covid['data'], format="%Y-%m-%dT%H:%M:%S.%f")
covid['data'] = covid['data'].apply(lambda x: x.strftime("%Y-%m-%d"))

In [15]:
# Dades centres

centres = pd.read_json('https://analisi.transparenciacatalunya.cat/resource/3u9c-b74b.json',
                      dtype={'codi_centre':str})

centres = centres.loc[centres.nom_municipi=="Sant Feliu de Llobregat", ['codi_centre','nom_municipi']]

In [16]:
# Unim dades

dades = pd.merge(covid, centres, on="codi_centre")

In [17]:
# Agrupem dades

dades = dades.groupby(['data'], as_index=False).agg({'alumn_positiu_acum':sum,
                                                     'personal_positiu_acum':sum,
                                                     'altres_positiu_acum':sum,
                                                     'grup_confin':sum})

In [18]:
# Calculem els nous positius diaris (desfem els valors acumulats)

for i in range(len(dades)-1,0,-1):
    dades.loc[i,'alumn_positiu_acum'] = dades.loc[i,'alumn_positiu_acum'] - dades.loc[i-1,'alumn_positiu_acum']
    dades.loc[i,'personal_positiu_acum'] = dades.loc[i,'personal_positiu_acum'] - dades.loc[i-1,'personal_positiu_acum']
    dades.loc[i,'altres_positiu_acum'] = dades.loc[i,'altres_positiu_acum'] - dades.loc[i-1,'altres_positiu_acum']
    dades.loc[i,'grup_confin'] = dades.loc[i,'grup_confin'] - dades.loc[i-1,'grup_confin']

dades = dades.rename(columns={'alumn_positiu_acum':'pos_alumn',
                              'personal_positiu_acum':'pos_pers',
                              'altres_positiu_acum':'pos_altres'})

In [24]:
dades.head()

,data,pos_alumn,pos_pers,pos_altres,grup_confin
0,2020-09-23,5,3,0,0.0
1,2020-09-24,0,0,0,3.0
2,2020-09-25,0,0,0,0.0
3,2020-09-26,0,0,0,0.0
4,2020-09-28,1,0,0,0.0
